In [1]:
from time import time
import re
import string
import random

algorytm naiwny

In [2]:
def naive_string_matching(text, pattern, printt = True):
    t = time()
    for s in range(0, len(text) - len(pattern) + 1):
        if pattern == text[s:s + len(pattern)] and printt:
            print(f"Przesunięcie {s} jest poprawne")
    return time() - t

algorytm automatu skończonego

In [190]:
def fa_string_matching(text, pattern, printt = True, res = []):
    tt = time()
    q = 0
    delta = transition_table(pattern)

    t = time()
    
    for s in range(len(text)):
        q = delta[q].get(text[s], 0) # jesli nie ma znaku w tablicy przejscia to domyslnie wrzuca nam 0

        if (q == len(delta) - 1):
            if printt:
                print(f"Przesunięcie {s + 1 - q} jest poprawne")
            # s + 1 - ponieważ przeczytaliśmy znak o indeksie s, więc przesunięcie jest po tym znaku
            res.append(s+1-q)
    return time() - tt, time() - t 


def check_pattern(pattern, text):
    return pattern == text[(len(text) - len(pattern)):]
#     print(pattern, text)
#     if (len(text) < len(pattern)):
#         return False
    
#     for i in range (len(pattern)):
#         if pattern[-i-1] != text[-i-1]:
#             return False
#     return True


def transition_table(pattern):
    result = []

    alphabet = set(pattern)

    for q in range(len(pattern) + 1):
        result.append({})
        for a in alphabet:
            k = min(len(pattern), q + 1)
            while True:            
#                 if re.search(f"{pattern[:k]}$", pattern[:q] + a):
#                     break
#                 print("pattern", pattern[:k], "text", pattern[:q] +a)
                
                if check_pattern(pattern[:k], pattern[:q] + a):
                    break
                    
                k = k - 1

            result[q][a] = k
    return result

In [191]:
from pprint import pprint

In [192]:
pattern = "ABAC"
s = time()
pprint(transition_table(pattern))
print(time() - s)

[{'A': 1, 'B': 0, 'C': 0},
 {'A': 1, 'B': 2, 'C': 0},
 {'A': 3, 'B': 0, 'C': 0},
 {'A': 1, 'B': 2, 'C': 4},
 {'A': 1, 'B': 0, 'C': 0}]
0.0004892349243164062


In [175]:
# pattern = ''.join(random.choice("aaaaaaab") for i in range(500))

In [181]:

# s = time()
# transition_table(pattern)
# print(time() - s)

0.04096198081970215


algorytm kmp (Knutha-Morrisa-Pratta)

In [48]:
def kmp_string_matching(text, pattern, printt = True, res = []):
    tt = time()
    pi = prefix_function(pattern)
    t = time()
    q = 0
    
    for i in range(len(text)):
        while q != 0 and pattern[q] != text[i]:
            q = pi[q - 1]

        if pattern[q] == text[i]:
            q = q + 1
        if q == len(pattern):
            if printt:
                print(f"Przesunięcie {i + 1 - q} jest poprawne")
            res.append(i+1-q)
            q = pi[q - 1]
    return time() - tt, time() - t

def prefix_function(pattern):
    pi = [0] * len(pattern)
    k = 0
    for q in range(1, len(pattern)):
        while k != 0 and pattern[k] != pattern[q]:
            k = pi[k - 1]

        if pattern[k] == pattern[q]:
            k = k + 1
        pi[q] = k
    return pi

Porównanie algorytmów

In [49]:
def speed_tests(text, pattern):
    naive = naive_string_matching(text, pattern, False)
    print("naive:        ", naive)
    print()
    print("fa")
    fa_total, fa_matching = fa_string_matching(text, pattern, False)
    print("total:        ", fa_total)
    print("matching:     ", fa_matching)
    print("preprocessing:", fa_total - fa_matching)
    fa_pre = fa_total - fa_matching
    
    print()
    print("kmp")
    kmp_total, kmp_matching = kmp_string_matching(text, pattern, False)
    print("total:        ", kmp_total)
    print("matching:     ", kmp_matching)
    print("preprocessing:", kmp_total - kmp_matching)
    kmp_pre = kmp_total - kmp_matching
    
    print()
    print("naive / fa matching: ", naive / fa_matching)
    print("naive / kmp matching:", naive / kmp_matching)
    print()
    print("fa preprocessing / kmp preprocessing:", fa_pre/kmp_pre)

In [193]:
text = open("ustawa", encoding="utf8").read()
pattern = "art"

In [194]:
kmp, fa = [], []
kmp_string_matching(text, pattern, False, kmp)
fa_string_matching(text, pattern, False, fa)

print(kmp == fa)

True


In [195]:
speed_tests(text, pattern)

naive:         0.03812885284423828

fa
total:         0.03319287300109863
matching:      0.03316783905029297
preprocessing: 2.5033950805664062e-05

kmp
total:         0.024707317352294922
matching:      0.0247039794921875
preprocessing: 3.337860107421875e-06

naive / fa matching:  1.1495730182006383
naive / kmp matching: 1.543429586164299

fa preprocessing / kmp preprocessing: 7.5


In [196]:
text = "a"*100
pattern = "aaaaaaabcdeaaaaaaafg"
# print(text, pattern)
speed_tests(text, pattern)

naive:         1.1920928955078125e-05

fa
total:         0.0005171298980712891
matching:      1.239776611328125e-05
preprocessing: 0.0005047321319580078

kmp
total:         2.5510787963867188e-05
matching:      1.9073486328125e-05
preprocessing: 6.4373016357421875e-06

naive / fa matching:  0.9615384615384616
naive / kmp matching: 0.625

fa preprocessing / kmp preprocessing: 78.4074074074074


In [197]:
text = ''.join(random.choice("0123456789") for i in range(100000))
# pattern = ''.join(random.choice("aaaaaaab") for i in range(10))
# text = "aaaaaabcccc"*1000000
pattern = ''.join([str(i) for i in range(100)])
# print(len(pattern))
# print(pattern)

# print()
# print(text)
n = 3000
pattern = "a" * (2*n)
pattern = "a" *n
speed_tests(text, pattern)

naive:         0.023135900497436523

fa
total:         0.01816868782043457
matching:      0.015487909317016602
preprocessing: 0.0026807785034179688

kmp
total:         0.012453079223632812
matching:      0.012074947357177734
preprocessing: 0.0003781318664550781

naive / fa matching:  1.4938039746925078
naive / kmp matching: 1.9160249575484738

fa preprocessing / kmp preprocessing: 7.0895334174022695
